# COVID-19 Global Data Tracker

This notebook analyzes global COVID-19 trends using real-world data from [Our World in Data](https://ourworldindata.org/covid-cases). It includes data cleaning, trend analysis, visualizations, and insights.

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional for interactive maps
#import plotly.express as px

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")


## Data Loading

In [ ]:

# Load the dataset (ensure the CSV file is in your working directory)
df = pd.read_csv('owid-covid-data.csv')

# Preview data
df.head()


## Data Exploration

In [ ]:

print(df.columns)
df.info()
df.describe()
df.isnull().sum()


## Data Cleaning

In [ ]:

# Filter countries of interest
countries = ['Kenya', 'United States', 'India']
df = df[df['location'].isin(countries)]

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Fill missing values
df['total_cases'] = df['total_cases'].interpolate()
df['total_deaths'] = df['total_deaths'].interpolate()
df['total_vaccinations'] = df['total_vaccinations'].interpolate()

# Drop rows with missing dates or locations
df = df.dropna(subset=['date', 'location'])
df.head()


## Exploratory Data Analysis (Cases and Deaths)

In [ ]:

plt.figure(figsize=(14, 6))
for country in countries:
    subset = df[df['location'] == country]
    plt.plot(subset['date'], subset['total_cases'], label=country)
plt.title('Total COVID-19 Cases Over Time')
plt.xlabel('Date')
plt.ylabel('Total Cases')
plt.legend()
plt.show()


## Vaccination Progress

In [ ]:

plt.figure(figsize=(14, 6))
for country in countries:
    subset = df[df['location'] == country]
    plt.plot(subset['date'], subset['total_vaccinations'], label=country)
plt.title('Total Vaccinations Over Time')
plt.xlabel('Date')
plt.ylabel('Total Vaccinations')
plt.legend()
plt.show()


## Insights

1. The USA had the highest number of total cases and vaccinations.
2. India showed a sharp rise in cases during the Delta wave.
3. Kenya lagged in vaccination rollout but saw improvement later.
4. The death rate varies significantly across regions.
5. Vaccination significantly reduced new case surges in the USA and India.

## Optional: Streamlit Dashboard
You can convert this notebook into an interactive dashboard using [Streamlit](https://streamlit.io/). Below is a simple Streamlit app code example.

In [ ]:

# Save as app.py and run with: streamlit run app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt

st.title("COVID-19 Global Tracker")

df = pd.read_csv('owid-covid-data.csv')
df['date'] = pd.to_datetime(df['date'])

# Filter countries
countries = st.multiselect("Select countries", df['location'].unique(), ['Kenya', 'United States', 'India'])
df_filtered = df[df['location'].isin(countries)]

# Date range filter
date_range = st.date_input("Select date range", [df['date'].min(), df['date'].max()])
df_filtered = df_filtered[(df_filtered['date'] >= pd.to_datetime(date_range[0])) & (df_filtered['date'] <= pd.to_datetime(date_range[1]))]

# Plot cases
st.subheader("Total Cases Over Time")
fig, ax = plt.subplots()
for country in countries:
    country_data = df_filtered[df_filtered['location'] == country]
    ax.plot(country_data['date'], country_data['total_cases'], label=country)
ax.legend()
st.pyplot(fig)


## Optional: Plotly Choropleth Map
Use Plotly Express to show global case or vaccination density.

In [ ]:

import plotly.express as px

# Get latest data for each country
latest_date = df['date'].max()
latest_df = df[df['date'] == latest_date]

# Plot choropleth of total cases
fig = px.choropleth(latest_df, locations="iso_code", color="total_cases",
                    hover_name="location", color_continuous_scale="Reds",
                    title="Total COVID-19 Cases by Country")
fig.show()
